<a href="https://colab.research.google.com/github/SantiagoSaldanaS/influence/blob/main/Densidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point
import folium
import zipfile
import os
import joblib


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Importación de datos DENUE 2025 de Querétaro

with zipfile.ZipFile("/content/denue_22_csv.zip", "r") as zip_ref:
    zip_ref.extractall("/content/denue_qro")

denue = pd.read_csv("/content/denue_qro/conjunto_de_datos/denue_inegi_22_.csv",
                    encoding="latin-1",
                    quotechar='"',
                    low_memory=False,
                    on_bad_lines="skip")

print("Filas:", len(denue))

Filas: 107843


In [ ]:
# Importación de datos ENIGH 2022 de Querétaro

with zipfile.ZipFile("/content/conjunto_de_datos_enigh_ns_2022_csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/enigh2022")


# Vamos a usar concentradohogar.csv como punto de nivel socioeconómico.
path = "/content/enigh2022/conjunto_de_datos_concentradohogar_enigh2022_ns/conjunto_de_datos/conjunto_de_datos_concentradohogar_enigh2022_ns.csv"
enigh = pd.read_csv(path, encoding="latin-1", low_memory=False)

print(enigh.columns.tolist())

enigh.head()

['ï»¿folioviv', 'foliohog', 'ubica_geo', 'tam_loc', 'est_socio', 'est_dis', 'upm', 'factor', 'clase_hog', 'sexo_jefe', 'edad_jefe', 'educa_jefe', 'tot_integ', 'hombres', 'mujeres', 'mayores', 'menores', 'p12_64', 'p65mas', 'ocupados', 'percep_ing', 'perc_ocupa', 'ing_cor', 'ingtrab', 'trabajo', 'sueldos', 'horas_extr', 'comisiones', 'aguinaldo', 'indemtrab', 'otra_rem', 'remu_espec', 'negocio', 'noagrop', 'industria', 'comercio', 'servicios', 'agrope', 'agricolas', 'pecuarios', 'reproducc', 'pesca', 'otros_trab', 'rentas', 'utilidad', 'arrenda', 'transfer', 'jubilacion', 'becas', 'donativos', 'remesas', 'bene_gob', 'transf_hog', 'trans_inst', 'estim_alqu', 'otros_ing', 'gasto_mon', 'alimentos', 'ali_dentro', 'cereales', 'carnes', 'pescado', 'leche', 'huevo', 'aceites', 'tuberculo', 'verduras', 'frutas', 'azucar', 'cafe', 'especias', 'otros_alim', 'bebidas', 'ali_fuera', 'tabaco', 'vesti_calz', 'vestido', 'calzado', 'vivienda', 'alquiler', 'pred_cons', 'agua', 'energia', 'limpieza', 'cu

,ï»¿folioviv,foliohog,ubica_geo,tam_loc,est_socio,est_dis,upm,factor,clase_hog,sexo_jefe,...,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,smg
0,100005002,1,1001,1,4,3,1,206,3,2,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,15558.3
1,100005003,1,1001,1,4,3,1,206,2,1,...,0.0,0.0,0.0,19565.21,0.0,0.0,0.0,0.0,0.0,15558.3
2,100005004,1,1001,1,4,3,1,206,2,1,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,15558.3
3,100012002,1,1001,1,3,2,2,167,3,1,...,0.0,0.0,0.0,28124.99,0.0,0.0,0.0,0.0,0.0,15558.3
4,100012002,2,1001,1,3,2,2,167,1,1,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,15558.3


In [ ]:
enigh["est_socio"].value_counts(dropna=False)

,count
est_socio,
2,47308
1,21142
3,15772
4,5880


In [ ]:
denue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107843 entries, 0 to 107842
Data columns (total 42 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          107843 non-null  int64  
 1   clee        107843 non-null  object 
 2   nom_estab   107843 non-null  object 
 3   raz_social  21753 non-null   object 
 4   codigo_act  107843 non-null  int64  
 5   nombre_act  107843 non-null  object 
 6   per_ocu     107843 non-null  object 
 7   tipo_vial   107786 non-null  object 
 8   nom_vial    107837 non-null  object 
 9   tipo_v_e_1  106598 non-null  object 
 10  nom_v_e_1   106581 non-null  object 
 11  tipo_v_e_2  106599 non-null  object 
 12  nom_v_e_2   106583 non-null  object 
 13  tipo_v_e_3  106598 non-null  object 
 14  nom_v_e_3   106373 non-null  object 
 15  numero_ext  99405 non-null   float64
 16  letra_ext   53191 non-null   object 
 17  edificio    14165 non-null   object 
 18  edificio_e  14263 non-null   object 
 19  nu

In [ ]:
# Limpieza de datos

# Eliminar filas sin coordenadas
denue_clean = denue.dropna(subset=["latitud", "longitud"])

# Convertir a float
denue_clean["latitud"] = denue_clean["latitud"].astype(float)
denue_clean["longitud"] = denue_clean["longitud"].astype(float)

# Convertir a GeoDataFrame
geometry = [Point(xy) for xy in zip(denue_clean["longitud"], denue_clean["latitud"])]
gdf_denue = gpd.GeoDataFrame(denue_clean, geometry=geometry, crs="EPSG:4326")

In [ ]:
denue_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107843 entries, 0 to 107842
Data columns (total 42 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          107843 non-null  int64  
 1   clee        107843 non-null  object 
 2   nom_estab   107843 non-null  object 
 3   raz_social  21753 non-null   object 
 4   codigo_act  107843 non-null  int64  
 5   nombre_act  107843 non-null  object 
 6   per_ocu     107843 non-null  object 
 7   tipo_vial   107786 non-null  object 
 8   nom_vial    107837 non-null  object 
 9   tipo_v_e_1  106598 non-null  object 
 10  nom_v_e_1   106581 non-null  object 
 11  tipo_v_e_2  106599 non-null  object 
 12  nom_v_e_2   106583 non-null  object 
 13  tipo_v_e_3  106598 non-null  object 
 14  nom_v_e_3   106373 non-null  object 
 15  numero_ext  99405 non-null   float64
 16  letra_ext   53191 non-null   object 
 17  edificio    14165 non-null   object 
 18  edificio_e  14263 non-null   object 
 19  nu

In [ ]:
# Entrenamiento de datos supervizado

# Variables que consideramos determinan el nivel socioeconómico.
features = [
    'ing_cor', 'ingtrab', 'sueldos', 'negocio', 'transfer', 'remesas',
    'gasto_mon', 'alimentos', 'vivienda', 'educacion',
    'agua', 'energia', 'comunica', 'cuidados', 'medicinas',
    'educa_jefe', 'edad_jefe',
    'tot_integ', 'hombres', 'mujeres', 'mayores', 'menores'
]

# Variable a comparar
target = 'est_socio'

# Bajo → Medio bajo → Medio alto → Alto  == 1 → 2 → 3 → 4

df_modelo = enigh[features + [target]].dropna()

# Selección de variables (features) y target
X = df_modelo[features]
y = df_modelo[target]

# Escalar variables ara ingresos/gastos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modelo de regresión logistica
model = LogisticRegression(max_iter=1000, solver='lbfgs')
model.fit(X_train, y_train)

# Evaluación
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

joblib.dump(model, 'modelo_nse_enigh.pkl')
joblib.dump(scaler, 'escalador_nse_enigh.pkl')

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


[[1954 2208   26    3]
 [1073 8041  379   40]
 [  43 2523  545   56]
 [   3  682  334  111]]
              precision    recall  f1-score   support

           1       0.64      0.47      0.54      4191
           2       0.60      0.84      0.70      9533
           3       0.42      0.17      0.24      3167
           4       0.53      0.10      0.17      1130

    accuracy                           0.59     18021
   macro avg       0.55      0.40      0.41     18021
weighted avg       0.57      0.59      0.55     18021



['escalador_nse_enigh.pkl']

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
print("RandomForestClassifier Report:")
print(classification_report(y_test, y_pred_rf))

gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

y_pred_gb = gb_model.predict(X_test)
print("\nGradientBoostingClassifier Report:")
print(classification_report(y_test, y_pred_gb))

svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
print("\nSupport Vector Machine Report:")
print(classification_report(y_test, y_pred_svm))

RandomForestClassifier Report:
              precision    recall  f1-score   support

           1       0.69      0.48      0.56      4191
           2       0.62      0.85      0.71      9533
           3       0.45      0.22      0.29      3167
           4       0.49      0.19      0.27      1130

    accuracy                           0.61     18021
   macro avg       0.56      0.43      0.46     18021
weighted avg       0.59      0.61      0.58     18021


GradientBoostingClassifier Report:
              precision    recall  f1-score   support

           1       0.68      0.49      0.57      4191
           2       0.62      0.84      0.71      9533
           3       0.46      0.22      0.30      3167
           4       0.46      0.22      0.30      1130

    accuracy                           0.61     18021
   macro avg       0.55      0.44      0.47     18021
weighted avg       0.60      0.61      0.58     18021


Support Vector Machine Report:
              precision    reca

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                              param_grid=param_grid_rf,
                              scoring='precision_weighted',
                              cv=3,
                              n_jobs=-1)

grid_search_rf.fit(X_train, y_train)

print("Best hyperparameters for RandomForestClassifier:", grid_search_rf.best_params_)
print("Best weighted precision for RandomForestClassifier:", grid_search_rf.best_score_)

best_rf_model = RandomForestClassifier(**grid_search_rf.best_params_, random_state=42)
best_rf_model.fit(X_train, y_train)

y_pred_best_rf = best_rf_model.predict(X_test)
print("\nBest RandomForestClassifier Report:")
print(classification_report(y_test, y_pred_best_rf))

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best hyperparameters for RandomForestClassifier: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best weighted precision for RandomForestClassifier: 0.5930608800356172

Best RandomForestClassifier Report:
              precision    recall  f1-score   support

           1       0.70      0.46      0.55      4191
           2       0.61      0.86      0.72      9533
           3       0.45      0.21      0.28      3167
           4       0.49      0.18      0.27      1130

    accuracy                           0.61     18021
   macro avg       0.56      0.43      0.46     18021
weighted avg       0.60      0.61      0.57     18021



In [ ]:
# Visualización

# Mapa centrado en Querétaro
mapa = folium.Map(location=[20.6, -100.4], zoom_start=11)

# Mostrar solo los primeros 1000 puntos (por rendimiento)
for _, row in gdf_denue.head(1000).iterrows():
    folium.CircleMarker(
        location=[row["latitud"], row["longitud"]],
        radius=2,
        popup=row["nom_estab"],
        color="blue",
        fill=True,
        fill_opacity=0.7
    ).add_to(mapa)

mapa
